# Example 1 - Transfer entropy on binary data

_Copyright (C) 2024-, J.T. Lizier; Distributed under GNU General Public License v3_

This is a sample notebook to run to check that your installation works ok

In [ ]:
# Import relevant libraries and start the JVM:

from jpype import *
import numpy
import random
import os

if (not isJVMStarted()):
    # Add JIDT jar library to the path -- it should be two folders up from the location of this notebook.
    jarLocation = os.path.join(os.getcwd(), "..", "..", "infodynamics.jar");
    if (not(os.path.isfile(jarLocation))):
    	exit("infodynamics.jar not found (expected at " + os.path.abspath(jarLocation))
    # Start the JVM (add the "-Xmx" option with say 1024M if you get crashes due to not enough memory space)
    startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + jarLocation)

# Generate some random binary data.
sourceArray = [random.randint(0,1) for r in range(100)]
destArray = [0] + sourceArray[0:99]
sourceArray2 = [random.randint(0,1) for r in range(100)]

# Create a TE calculator and run it:
teCalcClass = JPackage("infodynamics.measures.discrete").TransferEntropyCalculatorDiscrete
teCalc = teCalcClass(2,1)
teCalc.initialise()

# First use simple arrays of ints, which we can directly pass in:
teCalc.addObservations(sourceArray, destArray)
print("For copied source, result should be close to 1 bit : %.4f" % teCalc.computeAverageLocalOfObservations())
teCalc.initialise()
teCalc.addObservations(sourceArray2, destArray)
print("For random source, result should be close to 0 bits: %.4f" % teCalc.computeAverageLocalOfObservations())

# Next, demonstrate how to do this with a numpy array
teCalc.initialise()
# Create the numpy arrays:
sourceNumpy = numpy.array(sourceArray, dtype=int)
destNumpy = numpy.array(destArray, dtype=int)
# The above can be passed straight through to JIDT in python 2:
# teCalc.addObservations(sourceNumpy, destNumpy)
# But you need to do this in python 3:
sourceNumpyJArray = JArray(JInt, 1)(sourceNumpy.tolist())
destNumpyJArray = JArray(JInt, 1)(destNumpy.tolist())
teCalc.addObservations(sourceNumpyJArray, destNumpyJArray)
print("Using numpy array for copied source, result confirmed as: %.4f" % teCalc.computeAverageLocalOfObservations())

print()
print("If you've got no error messages, then your JIDT set-up is ok!");